# Tagger microscope 2023 rate scan analysis

This is an alternate version of my tagm_timing notebook that uses RDataFrame queries instead of treeview.fill_histograms for tree data analysis.

The following runs taken in winter 2023 contain useful information to answer questions related to the relative performance of the tagger microscope over a wide range in beam intensities.

1) Run 120827 - mode 9, 30nA, 9M events
2) Run 120597 - mode 9, 300nA, 76M events
3) Run 120595 - mode 9, 400nA, 200M events
4) Run 120593 - mode 9, 600nA, 150M events
5) Run 120589 - mode 9, 700nA, 73M events
6) Run 120582 - mode 9, 900nA, 185M events

In [1]:
from pyxrootd import client as xclient
from gluex.jupyroot.treeview import treeview
import ROOT
%jsroot on
import numpy as np
import os
os.chdir("/srv/jupyter/TAGM studies")
#os.remove("tagm_timing.root")

In [2]:
import dask.distributed
import dask
dclient = dask.distributed.Client(n_workers=50, threads_per_worker=1, dashboard_address='0.0.0.0:8789')

In [3]:
xrdurl = "root://nod29.phys.uconn.edu/"
xrdpath = "/Gluex/beamline/PStags-1-2023/"
scansteps = {"30nA": 120827, "300nA": 120597, "400nA": 120595,
             "600nA": 120593, "700nA": 120589, "900nA": 120582}
nfiles = {"30nA": 100, "300nA": 5, "400nA": 5,
          "600nA": 5, "700nA": 5, "900nA": 50}
xfs = xclient.FileSystem(xrdurl)
status, listing = xfs.dirlist(xrdpath)

chain = {}
tview = {}
for cur,run in scansteps.items():
    chain[cur] = ROOT.TChain("pstags", "")
    nfile = 0
    for entry in listing['dirlist']:
        if f"PStagstudy2_{run}_" in entry['name']:
            chain[cur].Add(xrdurl + xrdpath + entry['name'])
            nfile += 1
            if nfile == nfiles[cur]:
                break
    print("chain for cur=", cur, "has nfile=", nfile)
    tview[cur] = treeview(chain[cur], f"tagm_timing.root/{cur}")
    tview[cur].enable_dask_cluster(dclient)

chain for cur= 30nA has nfile= 66
chain for cur= 300nA has nfile= 5
chain for cur= 400nA has nfile= 5
chain for cur= 600nA has nfile= 5
chain for cur= 700nA has nfile= 5
chain for cur= 900nA has nfile= 50


# list the contents of the PStagstudy pstags tree

In [4]:
ftree = ROOT.TFile.Open(xrdurl + xrdpath + "PStagstudy2_120582_100.root")
pstags = ftree.Get("pstags")
pstags.Print()

******************************************************************************
*Tree    :pstags    : PS tag study                                           *
*Entries :  1443920 : Total =     17440332408 bytes  File  Size = 4317848895 *
*        :          : Tree compression factor =   4.04                       *
******************************************************************************
*Br    0 :runno     : runno/i                                                *
*Entries :  1443920 : Total  Size=    5790179 bytes  File Size  =      45038 *
*Baskets :      151 : Basket Size=      38912 bytes  Compression= 128.49     *
*............................................................................*
*Br    1 :eventno   : eventno/i                                              *
*Entries :  1443920 : Total  Size=    5790489 bytes  File Size  =    2440825 *
*Baskets :      151 : Basket Size=      38912 bytes  Compression=   2.37     *
*...................................................

In [5]:
def tagm_time_hinit():
    h = {}
    for col in range(102):
        hname = f"htagmt{col+1}"
        h[hname] = ROOT.TH1D(hname, f"tagm time - tRF for column {col+1}", 1000, -14, 14)
        h[hname].GetXaxis().SetTitle("t - tRF (ns)")
        h[hname].GetYaxis().SetTitle("counts")
    return h
def tagm_time_hfill(row, histos, beam_period_ns=4.008):
    #if row.beamcurrent < 800:
    #    return histos
    for i in range(row.nrf):
        if row.rf_sys[i] == 2048:
            trf = row.rf_time[i]
            trf -= np.round(trf / beam_period_ns) * beam_period_ns
    for i in range(row.ntagm):
        if row.tagm_has_tdc[i] == 0 or row.tagm_has_adc[i] == 0 or row.tagm_qf[i] != 0:
            continue
        col = row.tagm_channel[i]
        if col != 1 and col != 45:
            continue
        t = row.tagm_time[i] - trf
        if abs(t) < 70:
            t -= np.round(t / 7 / beam_period_ns) * 7 * beam_period_ns
            histos[f"htagmt{col}"].Fill(t)
    return histos

u = tview['900nA'].declare_histograms("tagm time", tagm_time_hinit, tagm_time_hfill)

In [6]:
tview['900nA'].fill_histograms(maxrows=1000000, chunksize=-100)
u = tview['900nA'].draw(["htagmt1", "htagmt45"], stats='nie')
htagmt45 = tview['900nA'].get('htagmt45')
htagmt45.SetTitle('')
htagmt45.SetMinimum(0)
htagmt45.SetStats(0)
tview['900nA'].current_canvas.cd(2)
htagmt45.Draw()

found 103 histograms that need filling, follow progress on dask monitor dashboard at http://cn447.storrs.hpc.uconn.edu:8789/status


/srv/.local/lib/python3.9/site-packages/distributed/client.py:3362: UserWarning: Sending large graph of size 9.87 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


fill_histograms read a total of 50 tree files, 76709920.0 records


In [7]:
hsets = ("tall", "tgood", "tadc", "ttdc", "tboth", "noped", "late", 
         "uflow", "oflow", "nofall", "nopeak", "badped", "qfok")

In [8]:
ROOT.EnableImplicitMT()

In [9]:
rdf = {}
for cur in scansteps:
    rdf[cur] = {"rdf": ROOT.RDataFrame(chain[cur])}
    rdf[cur]["nsets"] = rdf[cur]['rdf'].Define("nsets",
    f"""
    std::vector<int> nsets = {{0,0,0,0,0,0,0,0,0,0,0,0,0}};
    for (int i=0; i < ntagm; ++i) {{
        nsets[0] += 1;
        if (tagm_has_adc[i]) {{
            nsets[2] += 1;
            if (tagm_has_tdc[i]) {{
                nsets[4] += 1;
                nsets[3] += 1;
                if (tagm_qf[i] == 0) {{
                    nsets[1] += 1;
                }}
            }}
        }}
        else if (tagm_has_tdc[i])
            nsets[3] += 1;
        if (tagm_qf[i] & (1 << 0))
            nsets[5] += 1;
        if (tagm_qf[i] & (1 << 1))
            nsets[6] += 1;
        if (tagm_qf[i] & (1 << 2))
            nsets[8] += 1;
        if (tagm_qf[i] & (1 << 3))
            nsets[7] += 1;
        if (tagm_qf[i] & (1 << 4))
            nsets[9] += 1;
        if (tagm_qf[i] & (1 << 5))
            nsets[10] += 1;
        if (tagm_qf[i] & (1 << 6))
            nsets[11] += 1;
        if (tagm_qf[i] == 0)
            nsets[12] += 1;
    }}
    return nsets;
    """)
    rdf[cur]["nsetsf"] = rdf[cur]["nsets"].Filter(f"beamcurrent > {int(cur[:-2])} * 0.667")
    for nset,hset in enumerate(hsets):
        rdf[cur][f"nset{nset}"] = rdf[cur]["nsetsf"].Define(f"nset{nset}", f"nsets[{nset}];")
        rdf[cur][f"h{hset}"] = rdf[cur][f"nset{nset}"].Histo1D((f"h{hset}", f"TAGM tags per event, with {hset}", 201, -0.5, 200.5),
                                                                f"nset{nset}")
        tview[cur].register_histogram(f"h{hset}", rdf[cur][f"h{hset}"])
    rdf[cur]["hibeam"] = rdf[cur]['nsetsf'].Histo1D((f"hibeam", "beam current", 1001, -0.5, 1000.5), "beamcurrent")
    tview[cur].register_histogram("hibeam", rdf[cur]["hibeam"])

In [10]:
u = tview["30nA"].draw([["htall", "htall"],
                        ["htall", "htall"],
                        ["htall", "htall"],
                       ])
flavors = {'tadc': 2, 'ttdc': 3, 'tboth': 4}
panel = 1
for cur in scansteps:
    tview["30nA"].current_canvas.cd(panel)
    %time htall = tview[cur].evaluate(f"htall")
    htall.SetMaximum(htall.GetMaximum() * 1.8)
    htall.SetTitle(htall.GetTitle() + f" at {cur}")
    htall.Draw()
    for hset in flavors:
        h = tview[cur].evaluate(f"h{hset}")
        h.SetLineColor(flavors[hset])
        h.SetTitle(h.GetTitle() + f" at {cur}")
        h.Draw("same")
    panel += 1

CPU times: user 2min 7s, sys: 15.6 s, total: 2min 23s
Wall time: 26.6 s
CPU times: user 43.8 s, sys: 3.44 s, total: 47.3 s
Wall time: 4.41 s
CPU times: user 44 s, sys: 5.09 s, total: 49 s
Wall time: 8.85 s
CPU times: user 58.7 s, sys: 7.45 s, total: 1min 6s
Wall time: 10.9 s
CPU times: user 3min 52s, sys: 53.6 s, total: 4min 46s
Wall time: 2min 1s
CPU times: user 10min 11s, sys: 21 s, total: 10min 32s
Wall time: 15.3 s


In [11]:
u = tview["30nA"].draw([["htall", "htall"],
                        ["htall", "htall"],
                        ["htall", "htall"],
                       ])
panel = 1
meancur = {}
for cur in scansteps:
    tview["30nA"].current_canvas.cd(panel)
    h = tview[cur].evaluate("hibeam")
    meancur[cur] = (h.GetMean(), h.GetStdDev() / (h.GetEntries() + 1e-99)**0.5)
    h.SetTitle(h.GetTitle() + f" at {cur}")
    h.Draw()
    panel += 1

In [12]:
meantags = {}
for cur in scansteps:
    meantags[cur] = {}
    for hset in hsets:
        h = tview[cur].evaluate(f"h{hset}")
        meantags[cur][hset] = (h.GetMean(), h.GetStdDev() / h.Integral()**0.5)

In [13]:
tview["30nA"].draw(['htall', 'htall'])
hntagm_vs_cur = ROOT.TH1D("hntagm_vs_cur", "TAGM tags per event vs rate", 1, 0, 1000)
hntagm_vs_cur.GetXaxis().SetTitle("beam current (nA)")
hntagm_vs_cur.GetYaxis().SetTitle("TAGM tags found / nA")
tview["30nA"].current_canvas.cd()
hntagm_vs_cur.SetStats(0)
hntagm_vs_cur.SetMaximum(0.12)
hntagm_vs_cur.Draw()

colors = {"tall": 1, "tadc": 2, "ttdc": 3, "tboth": 4, "tgood": 6}
xnA = [meancur[cur][0] for cur in scansteps]
xnAerr = [meancur[cur][1] for cur in scansteps]
hgraphs = {hset: 0 for hset in hsets}
for hset in colors:
    yf = [meantags[cur][hset][0] / (meancur[cur][0] + 1e-99) for cur in scansteps]
    yferr = [meantags[cur][hset][1] / (meancur[cur][0] + 1e-99) for cur in scansteps]
    hgraphs[hset] = ROOT.TGraphErrors(len(xnA), np.array(xnA, dtype=float), np.array(yf, dtype=float),
                                           np.array(xnAerr, dtype=float), np.array(yferr, dtype=float))
    hgraphs[hset].SetLineColor(colors[hset])
    hgraphs[hset].SetMarkerStyle(3)
    hgraphs[hset].SetMarkerColor(colors[hset])
    hgraphs[hset].Draw("same p")
legend = ROOT.TLegend(0.82, 0.65, 1.0, 0.95)
legend.AddEntry(hgraphs['tall'], "all hits", "p")
legend.AddEntry(hgraphs['tadc'], "has fadc", "p")
legend.AddEntry(hgraphs['tadc'], "has tdc", "p")
legend.AddEntry(hgraphs['tboth'], "has both", "p")
legend.AddEntry(hgraphs['tgood'], "both and QF=0", "p")
legend.Draw()